In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')

In [33]:
df = pd.read_csv('data/preprocessed_0815.csv')
df.head()

,realized_volatility,seconds_count,seconds_right_skew,seconds_skewness,seconds_kurtosis,wap_skewness,wap_kurtosis,bid_size1_skewness,bid_size1_max,bid_size1_mean,ask_size1_skewness,ask_size1_max,ask_size1_mean,bid_size2_skewness,ask_size2_skewness,row_id,size_max,size_sum,order_count_max,target
0,0.004499,302.0,152.841727,0.510418,-1.181185,-0.853924,3.512603,-0.803095,500.0,78.264901,-0.246879,228.0,74.579470,-0.669089,-0.426180,0-5,499.0,3179.0,12.0,0.004136
1,0.001204,200.0,147.965217,-0.362378,-1.001272,0.091449,0.511806,0.288387,402.0,149.965000,1.345276,400.0,71.145000,1.269870,-0.172575,0-11,280.0,1289.0,6.0,0.001445
2,0.002369,188.0,159.897059,0.506732,-1.216982,-0.960020,-0.131697,-0.164424,304.0,96.132979,1.279381,449.0,131.037234,0.538682,-1.122790,0-16,391.0,2161.0,8.0,0.002168
3,0.002574,120.0,120.509434,-0.032548,-1.013334,0.692342,-0.116943,0.448781,370.0,114.458333,0.480632,400.0,120.800000,1.293843,0.790816,0-31,450.0,1962.0,15.0,0.002195
4,0.001894,176.0,131.988764,-0.402382,-1.151718,0.377272,-1.027612,0.579404,402.0,119.823864,0.121758,333.0,88.477273,-0.434153,1.519461,0-62,341.0,1791.0,17.0,0.001747


In [36]:
using_Xcols = ['realized_volatility', 'seconds_count',
       'seconds_right_skew', 'seconds_skewness', 'seconds_kurtosis',
       'wap_skewness', 'wap_kurtosis', 'bid_size1_skewness',
       'ask_size1_skewness', 'bid_size2_skewness', 'ask_size2_skewness',
       'size_max', 'size_sum', 'order_count_max']
using_cols = using_Xcols + ['row_id','target']

In [37]:
merged_df = df[using_cols]
merged_df.head()

,realized_volatility,seconds_count,seconds_right_skew,seconds_skewness,seconds_kurtosis,wap_skewness,wap_kurtosis,bid_size1_skewness,ask_size1_skewness,bid_size2_skewness,ask_size2_skewness,size_max,size_sum,order_count_max,row_id,target
0,0.004499,302.0,152.841727,0.510418,-1.181185,-0.853924,3.512603,-0.803095,-0.246879,-0.669089,-0.426180,499.0,3179.0,12.0,0-5,0.004136
1,0.001204,200.0,147.965217,-0.362378,-1.001272,0.091449,0.511806,0.288387,1.345276,1.269870,-0.172575,280.0,1289.0,6.0,0-11,0.001445
2,0.002369,188.0,159.897059,0.506732,-1.216982,-0.960020,-0.131697,-0.164424,1.279381,0.538682,-1.122790,391.0,2161.0,8.0,0-16,0.002168
3,0.002574,120.0,120.509434,-0.032548,-1.013334,0.692342,-0.116943,0.448781,0.480632,1.293843,0.790816,450.0,1962.0,15.0,0-31,0.002195
4,0.001894,176.0,131.988764,-0.402382,-1.151718,0.377272,-1.027612,0.579404,0.121758,-0.434153,1.519461,341.0,1791.0,17.0,0-62,0.001747


#### Train_test split

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, idx_train, idx_test = \
train_test_split(merged_df[using_Xcols], merged_df['target'], merged_df['row_id'], test_size=0.2)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(idx_train.shape)
print(idx_test.shape)

(343145, 14)
(85787, 14)
(343145,)
(85787,)
(343145,)
(85787,)


In [39]:
# import libraries
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error, r2_score

#### 1. XGB

In [40]:
# n_estimators, max_depth, min_child_weight 먼저
xgbr = XGBRegressor(objective = 'reg:squarederror')
n_estimators = list(range(100, 1000, 100))
max_depth = np.arange(3, 9, 2)
min_child_weight = np.arange(2, 6, 2)

param_grid = dict(n_estimators=n_estimators,
                  max_depth=max_depth,
                  min_child_weight=min_child_weight)

rd_search = RandomizedSearchCV(xgbr, param_grid, scoring = 'neg_mean_squared_error', 
                               verbose=1, cv=5)
rd_search.fit(X_train, y_train)
scores_df = pd.DataFrame(rd_search.cv_results_)
best_params = scores_df[scores_df['rank_test_score']==1]['params'][0]

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaco

C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaco

KeyError: 0

In [49]:
best_params = scores_df[scores_df['rank_test_score']==1]['params'].iloc[0]

In [50]:
best_params

{'n_estimators': 400, 'min_child_weight': 2, 'max_depth': 3}

In [53]:
# subsample, colsample_bytree, gamma, reg_alpha 적합
xgbr = XGBRegressor(objective = 'reg:squarederror',
                    n_estimators = 400,
                    min_child_weight = 2,
                    max_depth = 3)

param_grid = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}

rd_search = RandomizedSearchCV(xgbr, param_grid, scoring = 'neg_mean_squared_error', 
                               verbose=1, cv=5)
rd_search.fit(X_train, y_train)
scores_df = pd.DataFrame(rd_search.cv_results_)
best_params = scores_df[scores_df['rank_test_score']==1]['params'].iloc[0]
best_params

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaco

C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaco

{'subsample': 0.9, 'colsample_bytree': 0.9}

In [56]:
# gamma, reg_alpha 적합
xgbr = XGBRegressor(objective = 'reg:squarederror',
                    n_estimators = 400,
                    min_child_weight = 2,
                    max_depth = 3,
                    subsample = 0.9,
                    colsample_bytree = 0.9)

param_grid = {
    'gamma':[i/10.0 for i in range(0,5)],
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}

rd_search = RandomizedSearchCV(xgbr, param_grid, scoring = 'neg_mean_squared_error', 
                               verbose=1, cv=5)
rd_search.fit(X_train, y_train)
scores_df = pd.DataFrame(rd_search.cv_results_)
best_params_g_r = scores_df[scores_df['rank_test_score']==1]['params'].iloc[0]
best_params_g_r

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaco

C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaco

{'reg_alpha': 1, 'gamma': 0.0}

In [58]:
# learning_rate 적합
xgbr = XGBRegressor(objective = 'reg:squarederror',
                    n_estimators = 400,
                    min_child_weight = 2,
                    max_depth = 3,
                    subsample = 0.9,
                    colsample_bytree = 0.9,
                    gamma = 0,
                    reg_alpha = 1)

learning_rate = np.arange(0.3, 0, -0.05)
param_grid = dict(learning_rate = learning_rate)

rd_search = GridSearchCV(xgbr, param_grid, scoring = 'neg_mean_squared_error', 
                               verbose=1, cv=5)
rd_search.fit(X_train, y_train)
scores_df = pd.DataFrame(rd_search.cv_results_)
best_params_lr = scores_df[scores_df['rank_test_score']==1]['params'].iloc[0]

Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaco

In [59]:
print(best_params_lr)

{'learning_rate': 0.050000000000000044}


In [60]:
# final model
best_xgbr = XGBRegressor(objective = 'reg:squarederror',
                    learning_rate = 0.05,
                    n_estimators = 400,
                    min_child_weight = 2,
                    max_depth = 3,
                    subsample = 0.9,
                    colsample_bytree = 0.9,
                    gamma = 0,
                    reg_alpha = 1)

best_xgbr.fit(X_train, y_train)
y_pred = best_xgbr.predict(X_test)
print('RMSE :', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2 :', r2_score(y_test, y_pred))
y_test_df = pd.DataFrame({'y_test':y_test, 'y_pred':y_pred})
y_test_df['rmspe_before'] = ((y_test_df.y_test-y_test_df.y_pred)/y_test_df.y_test)**2
print('RMSPE :',np.sqrt(y_test_df.rmspe_before.sum()/len(y_test_df)))

RMSE : 0.0013660585247621214
R2 : 0.7841772720144007
RMSPE : 0.304783932310892
